# <center>Model calibration</center>

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, <a href="https://twitter.com/guerrero_oa">@guerrero_oa</a>)

In this tutorial I will calibrate the free parameters of PPI's model. First, I will load all the data that I have prepared in the previous tutorials. Then, I extract the relevant information and put it in adequate data structures. Finally, I run the calibration function and save the results with the parameter values.

## Importing Python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np

## Importing PPI functions

In this tutorial, I will import the PPI source code directly from its repository. This means that I will place a request to GitHub, download the `ppi.py` file, and copy it locally into the folder where these tutorials are saved. Then, I will import ppi. This approach is useful if you want to run this tutorial in a cloud computing service.

An alternative would be to manually copy the `ppi.py` file into the folder where this tutorial is located.

In [2]:
import requests # the Python library that helps placing requests to websites
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source_code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

## Load data

### Indicators

In [3]:
df_indis = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_indicators.csv')

N = len(df_indis) # number of indicators
I0 = df_indis.I0.values # initial values
IF = df_indis.IF.values # final values
success_rates = df_indis.successRates.values # success rates
R = df_indis.instrumental # instrumental indicators
qm = df_indis.qm.values # quality of monitoring
rl = df_indis.rl.values # quality of the rule of law
indis_index = dict([(code, i) for i, code in enumerate(df_indis.seriesCode)]) # used to build the network matrix

### Interdependency network

In [4]:
df_net = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_network.csv')

A = np.zeros((N, N)) # adjacency matrix
for index, row in df_net.iterrows():
    i = indis_index[row.origin]
    j = indis_index[row.destination]
    w = row.weight
    A[i,j] = w

### Budget

In [5]:
df_exp = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_expenditure.csv')

Bs = df_exp.values[:,1::] # disbursement schedule (assumes that the expenditure programmes are properly sorted)

### Budget-indicator mapping

In [6]:
df_rela = pd.read_csv('https://raw.githubusercontent.com/oguerrer/ppi/main/tutorials/clean_data/data_relational_table.csv')

B_dict = {} # PPI needs the relational table in the form of a Python dictionary
for index, row in df_rela.iterrows():
    B_dict[indis_index[row.seriesCode]] = [programme for programme in row.values[1::][row.values[1::].astype(str)!='nan']]

## Calibrate

Now I run the calibration function to show that it works. Before that, let me explain a couple of new inputs that the user needs to provide:

* <strong>threshold</strong>: How well should the model be fit.
* <strong>parallel_processes</strong>: The number of processes (workers) to be ran in parallel.
* <strong>verbose</strong>: Whether to print or not the outputs as the calibration progresses.
* <strong>low_precision_counts</strong>: The number of iterations that use few Monte Carlo simulations.

The <strong>threshold</strong> parameter indicates the quality of the goodness of fit. More specifically, how good should the worst-fitted indicator be. The best possible fit is close to 1, but cannot be exactly 1 due to the stochasticity of the model. The higher the threshold, the mode Monte Carlo simulations are needed and, thus, the more time and computational resources are needed to complete the calibration.

Parameter <strong>parallel_processes</strong> is used to enhance efficiency. Since each Monte Carlo simulation is independent of each other, this workload can be distributed across multiple cores or processors. Today, most personal devices have the capability of handling this distributed load, so here I show how to calibrate the model using 4 parallel processes. It is recommended that you know how many cores or processors your equipment has, and that <strong>parallel_processes</strong> does not exceed that number. Otherwise, the performance of the calibration may be sub-optimal.

Finally, the <strong>low_precision_counts</strong> parameter helps accelerating the calibration. At the beginning of the calibration, the algorithm proposes a random set of parameters for the model. Because this proposal is unrelated to the true parameters, the errors tend to be large. In the presence of large errors, one can improve the goodness of fit without needing too much precision in each evaluation (i.e., without running too many Monte Carlo simulations). Hence, this parameter determines how many low-precision iterations of the algorithm should be run before proceeding to the high-precision ones. This accelerates the calibration procedure substantially.

In [7]:
T = Bs.shape[1]
parallel_processes = 4 # number of cores to use
threshold = 0.6 # the quality of the calibration (I choose a medium quality for illustration purposes)
low_precision_counts = 50 # number of low-quality iterations to accelerate the calibration

parameters = ppi.calibrate(I0, IF, success_rates, A=A, R=R, qm=qm, rl=rl, Bs=Bs, B_dict=B_dict,
              T=T, threshold=threshold, parallel_processes=parallel_processes, verbose=True,
             low_precision_counts=low_precision_counts)

Iteration: 1 .    Worst goodness of fit: -884997.9999982293
Iteration: 2 .    Worst goodness of fit: -577124.9999988454
Iteration: 3 .    Worst goodness of fit: -258749.99999948233
Iteration: 4 .    Worst goodness of fit: -69046.87499986183
Iteration: 5 .    Worst goodness of fit: -16415.96874996715
Iteration: 6 .    Worst goodness of fit: -18878.906249962227
Iteration: 7 .    Worst goodness of fit: -9081.496093731826
Iteration: 8 .    Worst goodness of fit: -7633.300781234727
Iteration: 9 .    Worst goodness of fit: -1031.6051151677336
Iteration: 10 .    Worst goodness of fit: -2239.5629882767694
Iteration: 11 .    Worst goodness of fit: -1275.3666381810376
Iteration: 12 .    Worst goodness of fit: -1229.164123532697
Iteration: 13 .    Worst goodness of fit: -245.50137329052043
Iteration: 14 .    Worst goodness of fit: -409.83724594034214
Iteration: 15 .    Worst goodness of fit: -119.75190448736628
Iteration: 16 .    Worst goodness of fit: -222.1255302424755
Iteration: 17 .    Worst 

## Calibration outputs

The output of the calibration function is a matrix with the following columns:

* <strong>alpha</strong>: the parameters related to structural constraints
* <strong>alpha_prime</strong>: the parameters related to structural costs
* <strong>beta</strong>: the parameters related to the probability of success
* <strong>T</strong>: the number of simulation periods
* <strong>error_alpha</strong>: the errors associated to the parameters $\alpha$ and $\alpha'$
* <strong>error_beta</strong>: the errors associated to the parameters $\beta$
* <strong>GoF_alpha</strong>: the goodness-of-fit associated to the parameters $\alpha$ and $\alpha'$
* <strong>GoF_beta</strong>: the goodness-of-fit associated to the parameters $\beta$

The top row of this matrix contains the column names, so I just need to transform these data into a DataFrame to export it.

In [8]:
df_params = pd.DataFrame(parameters[1::], columns=parameters[0])

In [9]:
df_params

,alpha,alpha_prime,beta,T,error_alpha,error_beta,GoF_alpha,GoF_beta
0,0.0004160375084310876,0.0006508570919842643,0.2560677978821674,69,0.00032311062937473434,0.0002214162897034716,0.9914599271736543,0.9975644208132618
1,0.015186232316362649,0.006767448848394903,0.023827176944713032,nan,-0.004079931773181222,-0.0003386916855898439,0.9893482793896377,0.9932261662882031
2,5.0718790269478984e-05,9.592408982727756e-09,0.008935064040105732,nan,-1.450300745831079e-05,-0.0023095002567662393,0.9549906665086598,0.9745954971755714
3,0.0005598669035583465,4.748120418437077e-07,0.026805936696062037,nan,0.00021026788076483616,0.0018678857921385905,0.9824937521289827,0.9941295017961359
4,0.0031083730487796476,9.914938177115819e-08,0.0381706043767838,nan,-0.001327694255550682,-0.001977510004863514,0.9861080345061235,0.9956494779893003
...,...,...,...,...,...,...,...,...
67,0.00010288489146151233,1.1023655255482158e-05,0.30156840087434417,nan,7.7503001768231e-05,0.007573292365387285,0.9848033329866214,0.9907437537756377
68,0.0017225703081263938,3.5388228911014877e-06,0.07260178181306441,nan,9.365069796279935e-05,0.0011904047925987982,0.9964192380190694,0.9947622189125653
69,0.0013060222236043009,1.1564332969827137e-05,0.10260898931155919,nan,0.0001974295963703132,-0.0013522092516069506,0.9966874228796927,0.9981407122790404
70,0.007688246106591582,3.050820206121638e-05,0.08449902291376084,nan,-0.005018903005729269,-0.007865413739898464,0.983216242317258,0.9866892998247873


## Save parameters data

In [10]:
df_params.to_csv('clean_data/parameters.csv', index=False)